# Reading in data with help of lymphoseq

In [86]:
import lymphoseq as ls
import pandas as pd
from pathlib import Path
import polars as pl

In [87]:
seqs_to_remove = "data/D20210208D_1-overrepresented-sequences.txt"
output_loc = "results_man/cleaned"
samplesheet = pd.read_csv("data/collated_info.csv")
samplesheet.head()

,SAMPLE,LOC,cells,newname,shortname,genotype_short,sample_short
0,FIN_15625_AgXP_TCRB.tsv.gz,/Users/ania/Documents/DQ2DQ8/data_link/D202102...,E,FIN_15625,F15625,heteroDQ2DQ8,F15625E
1,FIN_15625_NAIVE_TCRB.tsv.gz,/Users/ania/Documents/DQ2DQ8/data_link/D202102...,N,FIN_15625,F15625,heteroDQ2DQ8,F15625N
2,FIN_16018_AgXP_TCRB.tsv.gz,/Users/ania/Documents/DQ2DQ8/data_link/D202102...,E,FIN_16018,F16018,homoDQ8,F16018E
3,FIN_16018_NAIVE_TCRB.tsv.gz,/Users/ania/Documents/DQ2DQ8/data_link/D202102...,N,FIN_16018,F16018,homoDQ8,F16018N
4,FIN_18072_AgXP_TCRB.tsv.gz,/Users/ania/Documents/DQ2DQ8/data_link/D202102...,E,FIN_18072,F18072,heteroDQ2DQ8,F18072E


In [88]:


row0 = samplesheet.iloc[0]
filepath = Path(row0['LOC']) / row0['SAMPLE']
sample_short = row0['sample_short']

sample_df = pd.read_csv(filepath, sep='\t', compression='gzip')
sample_df['sample_short'] = sample_short


print(sample_df.head())
print(sample_df.columns.tolist())

                                          nucleotide          aminoAcid  \
0  CACCTACACACCCTGCAGCCAGAAGACTCGGCCCTGTATCTCTGCG...      CASSQDKNQPQHF   
1  ATCCAGCGCACACAGCAGGAGGACTCGGCCGTGTATCTCTGTGCCA...     CASSLMSGPGELFF   
2  TCAGAACCCAGGGACTCAGCTGTGTACTTCTGTGCCAGCAGACGCC...  CASRRPTGQGIRSGYTF   
3  ATCAATTCCCTGGAGCTTGGTGACTCTGCTGTGTATTTCTGTGCCA...     CASSQGTGGSPLHF   
4  GATCCAGCGCACAGAGCAGGGGGACTCAGCTGTGTATCTCTGTGCC...                NaN   

   count (templates/reads)  frequencyCount (%)  cdr3Length  \
0                       14            0.000759          39   
1                        5            0.000271          42   
2                        9            0.000488          51   
3                        8            0.000434          42   
4                        4            0.000217          41   

          vMaxResolved vFamilyName         vGeneName  vGeneAllele vFamilyTies  \
0        TCRBV04-03*01     TCRBV04        TCRBV04-03          1.0         NaN   
1           TC

/var/folders/bj/8722bm4j7tj2r8twwzwxs5z00000gn/T/ipykernel_67064/694894960.py:5: DtypeWarning: Columns (0: vFamilyTies, 1: jGeneNameTies, 2: jGeneAlleleTies) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(filepath, sep='\t', compression='gzip')


In [89]:
# Sumamrise NaN per column
sample_df.isna().sum()[sample_df.isna().sum() > 0].sort_values(ascending=True)

jGeneName                                      6
jGeneAllele                                   11
vMaxResolved                                  19
vFamilyName                                   19
vGeneName                                   2148
aminoAcid                                   2527
dMaxResolved                                3849
dFamilyName                                 3849
dGeneName                                   3849
vGeneAllele                                 4620
dGeneAllele                                 5917
dFamilyTies                                12721
dGeneNameTies                              12721
vGeneAlleleTies                            13985
vGeneNameTies                              14309
dGeneAlleleTies                            14389
vFamilyTies                                16438
jGeneNameTies                              16451
jGeneAlleleTies                            16452
vAlignSubstitutionGeneThreePrimeIndexes    16457
vOrphon             

In [90]:
# Check how many templates we will lose if we take only solved ones

# Total sum of 'count (templates/reads)'
total_count = sample_df['count (templates/reads)'].sum()

# Group by whether jGeneName and vFamilyName are NA, and include sequenceStatus, compute the sum of 'count (templates/reads)'
group1 = sample_df.groupby([sample_df[['jGeneName', 'vFamilyName']].isna().any(axis=1), 'sequenceStatus'])['count (templates/reads)'].sum()

# Express as fractions of the total count
group1_fraction = (group1 / total_count) * 100
print("Grouping by jGeneName and vFamilyName being NA, and sequenceStatus (as fractions):")
print(group1_fraction)

# Group by whether jGeneName is NA, include sequenceStatus, compute the sum of 'count (templates/reads)'
group2 = sample_df.groupby([sample_df[['jGeneName','vGeneName']].isna().any(axis=1), 'sequenceStatus'])['count (templates/reads)'].sum()

# Express as fractions of the total count
group2_fraction = (group2 / total_count) * 100
print("Grouping by jGeneName/vGeneName being NA, and sequenceStatus (as fractions):")
print(group2_fraction)

# Collate the grouping results into a single DataFrame for better visualization
collated_results = pd.concat([
    group1_fraction.rename("Group1 Fraction"),
    group2_fraction.rename("Group2 Fraction")
], axis=1).reset_index()

print("Collated Results:")
print(collated_results)


Grouping by jGeneName and vFamilyName being NA, and sequenceStatus (as fractions):
       sequenceStatus
False  In                83.114772
       Out               15.292078
       Stop               1.452260
True   In                 0.081283
       Out                0.054189
       Stop               0.005419
Name: count (templates/reads), dtype: float64
Grouping by jGeneName/vGeneName being NA, and sequenceStatus (as fractions):
       sequenceStatus
False  In                72.282432
       Out               13.601387
       Stop               1.284274
True   In                10.913623
       Out                1.744879
       Stop               0.173404
Name: count (templates/reads), dtype: float64
Collated Results:
   level_0 sequenceStatus  Group1 Fraction  Group2 Fraction
0    False             In        83.114772        72.282432
1    False            Out        15.292078        13.601387
2    False           Stop         1.452260         1.284274
3     True             In 

In [91]:
# Read in the file seqs_to_remove, which contains a list of CDR3 sequences to remove from the data
seqs_to_remove_df = pd.read_csv(seqs_to_remove, sep='\t', names=['nucleotide', 'aminoAcid'], header=None)

# Remove sequences by matching their nucleotide and aminoAcid
merged_df = sample_df.merge(seqs_to_remove_df, on=['nucleotide', 'aminoAcid'], how='left', indicator=True)
removed_sequences = merged_df[merged_df['_merge'] == 'both']
remaining_sequences = merged_df[merged_df['_merge'] == 'left_only']

# Write info about how many sequences were found and removed
num_removed_rows = len(removed_sequences)
removed_counts_sum = removed_sequences['count (templates/reads)'].sum()
print(f"Number of sequences removed: {num_removed_rows}")
print(f"Sum of counts removed: {removed_counts_sum}")

# Take only rows where jGeneName and vFamilyName are not NA
filtered_df = remaining_sequences[~remaining_sequences[['jGeneName', 'vFamilyName']].isna().any(axis=1)]

# Split into productive and non-productive sequences
productive_df = filtered_df[filtered_df['sequenceStatus'] == 'In']
nonproductive_df = filtered_df[filtered_df['sequenceStatus'] != 'In']

# Save the outputs
productive_output_path = f"{output_loc}/{sample_short}_productive.tsv.gz"
nonproductive_output_path = f"{output_loc}/{sample_short}_nonproductive.tsv.gz"
productive_df.to_csv(productive_output_path, sep='\t', index=False, compression='gzip')
nonproductive_df.to_csv(nonproductive_output_path, sep='\t', index=False, compression='gzip')

print(f"Productive sequences saved to: {productive_output_path}")
print(f"Non-productive sequences saved to: {nonproductive_output_path}")

# Create a cleanup summary DataFrame
cleanup_summary = pd.DataFrame({
    'Category': ['Removed Sequences', 'Productive Sequences', 'Non-Productive Sequences'],
    'Num Sequences': [num_removed_rows, len(productive_df), len(nonproductive_df)],
    'Sum of Counts': [removed_counts_sum, productive_df['count (templates/reads)'].sum(), nonproductive_df['count (templates/reads)'].sum()]
})

# Save the cleanup summary
cleanup_summary_path = f"{output_loc}/{sample_short}_cleanup_summary.tsv"
cleanup_summary.to_csv(cleanup_summary_path, sep='\t', index=False)
print(f"Cleanup summary saved to: {cleanup_summary_path}")

Number of sequences removed: 2
Sum of counts removed: 3
Productive sequences saved to: results_man/cleaned/F15625E_productive.tsv.gz
Non-productive sequences saved to: results_man/cleaned/F15625E_nonproductive.tsv.gz
Cleanup summary saved to: results_man/cleaned/F15625E_cleanup_summary.tsv


In [92]:
# Print the first 5 rows of sample_df with all columns
pd.set_option('display.max_columns', None)  # Ensure all columns are displayed
print(sample_df.head())

                                          nucleotide          aminoAcid  \
0  CACCTACACACCCTGCAGCCAGAAGACTCGGCCCTGTATCTCTGCG...      CASSQDKNQPQHF   
1  ATCCAGCGCACACAGCAGGAGGACTCGGCCGTGTATCTCTGTGCCA...     CASSLMSGPGELFF   
2  TCAGAACCCAGGGACTCAGCTGTGTACTTCTGTGCCAGCAGACGCC...  CASRRPTGQGIRSGYTF   
3  ATCAATTCCCTGGAGCTTGGTGACTCTGCTGTGTATTTCTGTGCCA...     CASSQGTGGSPLHF   
4  GATCCAGCGCACAGAGCAGGGGGACTCAGCTGTGTATCTCTGTGCC...                NaN   

   count (templates/reads)  frequencyCount (%)  cdr3Length  \
0                       14            0.000759          39   
1                        5            0.000271          42   
2                        9            0.000488          51   
3                        8            0.000434          42   
4                        4            0.000217          41   

          vMaxResolved vFamilyName         vGeneName  vGeneAllele vFamilyTies  \
0        TCRBV04-03*01     TCRBV04        TCRBV04-03          1.0         NaN   
1           TC

In [93]:
# Summarise the content of the column 'sequenceStatus'
sequence_status_summary = sample_df['sequenceStatus'].value_counts()
print("Summary of 'sequenceStatus':")
print(sequence_status_summary)

Summary of 'sequenceStatus':
sequenceStatus
In      13688
Out      2527
Stop      242
Name: count, dtype: int64


In [94]:
# Calculate repertoire diversity metrics
diversity = ls.clonality(sample_df)

# Visualize clonal expansion
fig = ls.plot_clonality(sample_df)
fig.show()

ValueError: Missing required columns: ['repertoire_id', 'junction_aa', 'duplicate_count']

In [ ]:
data = ls.read_immunoseq(filepath )
print(data.columns)


Processing files: 100%|██████████| 1/1 [00:00<00:00, 12.92file/s, records=16,457, current=16,457]

✅ Completed: 1 files processed, 16,457 total records standardized
['sequence_id', 'repertoire_id', 'sequence', 'sequence_aa', 'junction', 'junction_aa', 'junction_length', 'junction_aa_length', 'v_call', 'd_call', 'j_call', 'c_call', 'productive', 'vj_in_frame', 'stop_codon', 'locus', 'duplicate_count', 'duplicate_frequency', 'consensus_count', 'v_family', 'd_family', 'j_family', 'cdr1', 'cdr1_aa', 'cdr2', 'cdr2_aa', 'cdr3', 'cdr3_aa', 'clone_id', 'cell_id', 'nucleotide', 'aminoAcid', 'count (templates/reads)', 'frequencyCount (%)', 'cdr3Length', 'vMaxResolved', 'vFamilyName', 'vGeneName', 'vGeneAllele', 'vFamilyTies', 'vGeneNameTies', 'vGeneAlleleTies', 'dMaxResolved', 'dFamilyName', 'dGeneName', 'dGeneAllele', 'dFamilyTies', 'dGeneNameTies', 'dGeneAlleleTies', 'jMaxResolved', 'jFamilyName', 'jGeneName', 'jGeneAllele', 'jFamilyTies', 'jGeneNameTies', 'jGeneAlleleTies', 'v_deletions', 'n1Insertion', 'd5_deletions', 'd3_deletions', 'n2Insertion', 'j_deletions', 'vIndex', 'n1Index', 'dIn

In [ ]:
def immunoseq_to_airr(df: pl.DataFrame) -> pl.DataFrame:
    """
    Rename ImmunoSEQ columns to AIRR Rearrangement standard column names and add a productive column.

    Args:
        df: Polars DataFrame with ImmunoSEQ-style columns.

    Returns:
        A new Polars DataFrame with AIRR standard column names and a productive column.
    """

    # Define the column mapping
    col_map = {
        "nucleotide": "sequence",
        "aminoAcid": "junction_aa",
        'cdr3Length':'junction_aa_length',
        "vGeneName": "v_call",
        "jGeneName": "j_call",
        "sequenceStatus": "frame_type",
        "count (templates/reads)": "duplicate_count",
        "vFamilyName": "v_family",
        "jFamilyName": "j_family",
        "vGeneAllele": "v_allele",
        "jGeneAllele": "j_allele",
        "dGeneName": "d_gene"
    }

    # Only keep mappings where the column exists in df
    existing_map = {old: new for old, new in col_map.items() if old in df.columns}
    print(f"Existing columns to rename: {list(existing_map.keys())}")

    target_cols = list(existing_map.values())
    cols_to_drop = [col for col in target_cols if col in df.columns]
    cols_to_drop += ["sequence_aa","junction","junction_length","d_call","c_call","productive","vj_in_frame","stop_codon","locus","duplicate_frequency","consensus_count","d_family","cdr1","cdr1_aa","cdr2","cdr2_aa","cdr3","cdr3_aa","clone_id","cell_id"]
    
    if cols_to_drop:
        print(f"Dropping existing columns to avoid conflicts: {cols_to_drop}")
        df = df.drop(cols_to_drop)

    # Perform renaming
    df = df.rename(existing_map)

   # Cast columns with 'length' or 'count' in the name to integers
    int_cols = [col for col in df.columns if "length" in col.lower() or "count" in col.lower()]
    if int_cols:
        print(f"Casting columns to integers: {int_cols}")
        df = df.with_columns([pl.col(col).cast(pl.Int64, strict=False) for col in int_cols])

    # Add a productive column based on frame_type
    if "frame_type" in df.columns:
        df = df.with_columns(
            (pl.col("frame_type") == "In").alias("productive")
        )

    return df

In [ ]:
airr_df = immunoseq_to_airr(data)
airr_df

Existing columns to rename: ['nucleotide', 'aminoAcid', 'cdr3Length', 'vGeneName', 'jGeneName', 'sequenceStatus', 'count (templates/reads)', 'vFamilyName', 'jFamilyName', 'vGeneAllele', 'jGeneAllele', 'dGeneName']
Dropping existing columns to avoid conflicts: ['sequence', 'junction_aa', 'junction_aa_length', 'v_call', 'j_call', 'duplicate_count', 'v_family', 'j_family', 'sequence_aa', 'junction', 'junction_length', 'd_call', 'c_call', 'productive', 'vj_in_frame', 'stop_codon', 'locus', 'duplicate_frequency', 'consensus_count', 'd_family', 'cdr1', 'cdr1_aa', 'cdr2', 'cdr2_aa', 'cdr3', 'cdr3_aa', 'clone_id', 'cell_id']
Casting columns to integers: ['duplicate_count', 'frequencyCount (%)', 'junction_aa_length', 'vAlignLength', 'vAlignSubstitutionCount']


sequence_id,repertoire_id,sequence,junction_aa,duplicate_count,frequencyCount (%),junction_aa_length,vMaxResolved,v_family,v_call,v_allele,vFamilyTies,vGeneNameTies,vGeneAlleleTies,dMaxResolved,dFamilyName,d_gene,dGeneAllele,dFamilyTies,dGeneNameTies,dGeneAlleleTies,jMaxResolved,j_family,j_call,j_allele,jFamilyTies,jGeneNameTies,jGeneAlleleTies,v_deletions,n1Insertion,d5_deletions,d3_deletions,n2Insertion,j_deletions,vIndex,n1Index,dIndex,n2Index,jIndex,frame_type,cloneResolved,vOrphon,dOrphon,jOrphon,vFunction,dFunction,jFunction,fractionNucleated,vAlignLength,vAlignSubstitutionCount,vAlignSubstitutionIndexes,vAlignSubstitutionGeneThreePrimeIndexes,vSeqWithMutations,productive,shannon_diversity
str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,bool,f64
"""""","""FIN_15625_AgXP_TCRB""","""CACCTACACACCCTGCAGCCAGAAGACTCG…","""CASSQDKNQPQHF""",14,null,39,"""TCRBV04-03*01""","""TCRBV04""","""TCRBV04-03""","""01""",null,null,null,null,null,null,null,"""TCRBD01,TCRBD02""","""TCRBD01-01,TCRBD02-01""",null,"""TCRBJ01-05*01""","""TCRBJ01""","""TCRBJ01-05""","""01""",null,null,null,"""0""","""2""","""5""","""5""","""0""","""4""","""42""","""59""","""61""","""-1""","""63""","""In""","""VDJ""",null,null,null,null,null,null,null,null,null,null,null,null,true,9.458566
"""""","""FIN_15625_AgXP_TCRB""","""ATCCAGCGCACACAGCAGGAGGACTCGGCC…","""CASSLMSGPGELFF""",5,null,42,"""TCRBV07-02""","""TCRBV07""","""TCRBV07-02""",null,null,null,"""01,02""","""TCRBD02-01""","""TCRBD02""","""TCRBD02-01""",null,null,null,"""01,02""","""TCRBJ02-02*01""","""TCRBJ02""","""TCRBJ02-02""","""01""",null,null,null,"""2""","""4""","""8""","""4""","""2""","""6""","""39""","""54""","""58""","""62""","""64""","""In""","""VDJ""",null,null,null,null,null,null,null,null,null,null,null,null,true,9.458566
"""""","""FIN_15625_AgXP_TCRB""","""TCAGAACCCAGGGACTCAGCTGTGTACTTC…","""CASRRPTGQGIRSGYTF""",9,null,51,"""TCRBV12""","""TCRBV12""",null,null,null,"""TCRBV12-03/12-04,TCRBV12-04""",null,"""TCRBD01-01*01""","""TCRBD01""","""TCRBD01-01""","""01""",null,null,null,"""TCRBJ01-02*01""","""TCRBJ01""","""TCRBJ01-02""","""01""",null,null,null,"""6""","""9""","""0""","""6""","""13""","""8""","""30""","""41""","""50""","""56""","""69""","""In""","""VDJ""",null,null,null,null,null,null,null,null,null,null,null,null,true,9.458566
"""""","""FIN_15625_AgXP_TCRB""","""ATCAATTCCCTGGAGCTTGGTGACTCTGCT…","""CASSQGTGGSPLHF""",8,null,42,"""TCRBV03-01/03-02*01""","""TCRBV03""","""TCRBV03-01/03-02""","""01""",null,null,null,"""TCRBD01-01*01""","""TCRBD01""","""TCRBD01-01""","""01""",null,null,null,"""TCRBJ01-06*02""","""TCRBJ01""","""TCRBJ01-06""","""02""",null,null,null,"""1""","""2""","""3""","""1""","""0""","""9""","""39""","""55""","""57""","""-1""","""65""","""In""","""VDJ""",null,null,null,null,null,null,null,null,null,null,null,null,true,9.458566
"""""","""FIN_15625_AgXP_TCRB""","""GATCCAGCGCACAGAGCAGGGGGACTCAGC…",null,4,null,41,"""TCRBV07-04*01""","""TCRBV07""","""TCRBV07-04""","""01""",null,null,null,"""TCRBD01-01*01""","""TCRBD01""","""TCRBD01-01""","""01""",null,null,null,"""TCRBJ01-02*01""","""TCRBJ01""","""TCRBJ01-02""","""01""",null,null,null,"""4""","""3""","""2""","""5""","""3""","""3""","""40""","""53""","""56""","""61""","""64""","""Out""","""VDJ""",null,null,null,null,null,null,null,null,null,null,null,null,false,9.458566
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""""","""FIN_15625_AgXP_TCRB""","""CATCAATTCCCTGGAGCTTGGTGACTCTGC…",null,1,null,41,"""TCRBV03-01/03-02*01""","""TCRBV03""","""TCRBV03-01/03-02""","""01""",null,null,null,"""TCRBD02-01""","""TCRBD02""","""TCRBD02-01""",null,null,null,"""01,02""","""TCRBJ02-02*01""","""TCRBJ02""","""TCRBJ02-02""","""01""",null,null,null,"""7""","""6""","""8""","""3""","""15""","""18""","""40""","""50""","""56""",""

In [ ]:
# Define a function to calculate Shannon diversity and return it as a single number
def compute_diversity_metrics(df: pl.DataFrame) -> pl.DataFrame:
    def shannon_diversity(df: pl.DataFrame, count_col="duplicate_count") -> float:
        if count_col in df.columns:
            total_count = df[count_col].sum()
            shannon_div = (
                -df
                .with_columns((pl.col(count_col) / total_count).alias("proportion"))
                .with_columns((pl.col("proportion") * pl.col("proportion").log()).alias("shannon_term"))
                ["shannon_term"]
                .sum()
            )
            return shannon_div
        return 0.0

    # Calculate Shannon diversity as a single number
    shannon_div = shannon_diversity(airr_df)

    # Calculate other diversity metrics
    diversity = ls.diversity_metrics(airr_df)
    isinstance(diversity, pl.DataFrame)

    # Add shannon_div as a new column to diversity (handles polars DataFrame, pandas DataFrame/Series, or scalars)
    diversity = diversity.with_columns(pl.lit(shannon_div).alias("shannon_diversity"))
    return diversity




In [ ]:
compute_diversity_metrics(airr_df)

repertoire_id,total_sequences,unique_productive_sequences,total_count,clonality,gini_coefficient,top_productive_sequence,convergence,shannon_diversity
str,i64,i64,i64,f64,f64,f64,f64,f64
"""FIN_15625_AgXP_TCRB""",13688,13541,15353,0.005772,0.101629,0.001238,1.010856,9.643172


In [ ]:
sample_df

,nucleotide,aminoAcid,count (templates/reads),frequencyCount (%),cdr3Length,vMaxResolved,vFamilyName,vGeneName,vGeneAllele,vFamilyTies,vGeneNameTies,vGeneAlleleTies,dMaxResolved,dFamilyName,dGeneName,dGeneAllele,dFamilyTies,dGeneNameTies,dGeneAlleleTies,jMaxResolved,jFamilyName,jGeneName,jGeneAllele,jFamilyTies,jGeneNameTies,jGeneAlleleTies,vDeletion,n1Insertion,d5Deletion,d3Deletion,n2Insertion,jDeletion,vIndex,n1Index,dIndex,n2Index,jIndex,estimatedNumberGenomes,sequenceStatus,cloneResolved,vOrphon,dOrphon,jOrphon,vFunction,dFunction,jFunction,fractionNucleated,vAlignLength,vAlignSubstitutionCount,vAlignSubstitutionIndexes,vAlignSubstitutionGeneThreePrimeIndexes,vSeqWithMutations,sample_short
0,CACCTACACACCCTGCAGCCAGAAGACTCGGCCCTGTATCTCTGCG...,CASSQDKNQPQHF,14,0.000759,39,TCRBV04-03*01,TCRBV04,TCRBV04-03,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"TCRBD01,TCRBD02","TCRBD01-01,TCRBD02-01",NaN,TCRBJ01-05*01,TCRBJ01,TCRBJ01-05,1.0,NaN,NaN,NaN,0,2,5,5,0,4,42,59,61,-1,63,14,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
1,ATCCAGCGCACACAGCAGGAGGACTCGGCCGTGTATCTCTGTGCCA...,CASSLMSGPGELFF,5,0.000271,42,TCRBV07-02,TCRBV07,TCRBV07-02,NaN,NaN,NaN,"01,02",TCRBD02-01,TCRBD02,TCRBD02-01,NaN,NaN,NaN,"01,02",TCRBJ02-02*01,TCRBJ02,TCRBJ02-02,1.0,NaN,NaN,NaN,2,4,8,4,2,6,39,54,58,62,64,5,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
2,TCAGAACCCAGGGACTCAGCTGTGTACTTCTGTGCCAGCAGACGCC...,CASRRPTGQGIRSGYTF,9,0.000488,51,TCRBV12,TCRBV12,NaN,NaN,NaN,"TCRBV12-03/12-04,TCRBV12-04",NaN,TCRBD01-01*01,TCRBD01,TCRBD01-01,1.0,NaN,NaN,NaN,TCRBJ01-02*01,TCRBJ01,TCRBJ01-02,1.0,NaN,NaN,NaN,6,9,0,6,13,8,30,41,50,56,69,9,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
3,ATCAATTCCCTGGAGCTTGGTGACTCTGCTGTGTATTTCTGTGCCA...,CASSQGTGGSPLHF,8,0.000434,42,TCRBV03-01/03-02*01,TCRBV03,TCRBV03-01/03-02,1.0,NaN,NaN,NaN,TCRBD01-01*01,TCRBD01,TCRBD01-01,1.0,NaN,NaN,NaN,TCRBJ01-06*02,TCRBJ01,TCRBJ01-06,2.0,NaN,NaN,NaN,1,2,3,1,0,9,39,55,57,-1,65,8,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
4,GATCCAGCGCACAGAGCAGGGGGACTCAGCTGTGTATCTCTGTGCC...,NaN,4,0.000217,41,TCRBV07-04*01,TCRBV07,TCRBV07-04,1.0,NaN,NaN,NaN,TCRBD01-01*01,TCRBD01,TCRBD01-01,1.0,NaN,NaN,NaN,TCRBJ01-02*01,TCRBJ01,TCRBJ01-02,1.0,NaN,NaN,NaN,4,3,2,5,3,3,40,53,56,61,64,4,Out,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16452,CATCAATTCCCTGGAGCTTGGTGACTCTGCTGTGTATTTCTGTGCC...,NaN,1,0.000054,41,TCRBV03-01/03-02*01,TCRBV03,TCRBV03-01/03-02,1.0,NaN,NaN,NaN,TCRBD02-01,TCRBD02,TCRBD02-01,NaN,NaN,NaN,"01,02",TCRBJ02-02*01,TCRBJ02,TCRBJ02-02,1.0,NaN,NaN,NaN,7,6,8,3,15,18,40,50,56,61,76,1,Out,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
16453,AAGATCCAGCCTGCAGAGCTTGGGGACTCGGCCGTGTATCTCTGTG...,CASSLDPDSPAFF,1,0.000054,39,TCRBV11-03*01,TCRBV11,TCRBV11-03,1.0,NaN,NaN,NaN,TCRBD02-01,TCRBD02,TCRBD02-01,NaN,NaN,NaN,"01,02",TCRBJ01-01*01,TCRBJ01,TCRBJ01-01,1.0,NaN,NaN,NaN,0,4,2,10,4,10,42,59,63,67,71,1,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
16454,CACATCAATTCCCTGGAGCTTGGTGACTCTGCTGTGTATTTCTGTG...,CASSLGGATEAFF,1,0.000054,39,TCRBV03-01/03-02*01,TCRBV03,TCRBV03-01/03-02,1.0,NaN,NaN,NaN,TCRBD02-01*01,TCRBD02,TCRBD02-01,1.0,NaN,NaN,NaN,TCRBJ01-01*01,TCRBJ01,TCRBJ01-01,1.0,NaN,NaN,NaN,4,1,8,0,1,4,42,55,56,64,65,1,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E
16455,AGTGACCAGTGCCCATCCTGAAGACAGCAGCTTCTACATCTGCAGT...,NaN,1,0.000054,41,TCRBV20,TCRBV20,NaN,NaN,NaN,"TCRBV20-01,TCRBV20-or09_02",NaN,NaN,NaN,NaN,NaN,"TCRBD01,TCRBD02","TCRBD01-01,TCRBD02-01",NaN,TCRBJ01-01*01,TCRBJ01,TCRBJ01-01,1.0,NaN,NaN,NaN,7,5,5,2,8,4,40,47,52,57,65,1,Out,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F15625E


In [ ]:
# Create a table summarizing the usage of vFamilyName, jGeneName, and cdr3Length
usage_summary = sample_df.groupby(['vFamilyName', 'jGeneName', 'cdr3Length']).size().reset_index(name='count')

# Arrange the table from highest to lowest count
usage_summary = usage_summary.sort_values(by='count', ascending=False)

# Display the table
usage_summary.head()

,vFamilyName,jGeneName,cdr3Length,count
535,TCRBV05,TCRBJ01-01,42,80
658,TCRBV05,TCRBJ02-01,45,72
827,TCRBV06,TCRBJ01-02,42,70
800,TCRBV06,TCRBJ01-01,42,67
564,TCRBV05,TCRBJ01-02,42,66


In [ ]:
# Create a table summarizing the usage of vFamilyName, jGeneName, and cdr3Length
usage_summary = sample_df.groupby(['vFamilyName', 'jFamilyName', 'cdr3Length']).size().reset_index(name='count')

# Arrange the table from highest to lowest count
usage_summary = usage_summary.sort_values(by='count', ascending=False)

# Display the table
usage_summary.head()

,vFamilyName,jFamilyName,cdr3Length,count
244,TCRBV05,TCRBJ02,45,271
202,TCRBV05,TCRBJ01,42,249
277,TCRBV06,TCRBJ01,42,243
205,TCRBV05,TCRBJ01,45,222
241,TCRBV05,TCRBJ02,42,219


In [ ]:
# Compute summaries for vFamilyName, jFamilyName, cdr3Length, and jGeneName
summary_params = ['vFamilyName', 'jFamilyName', 'cdr3Length', 'jGeneName']
all_summaries = []

for param in summary_params:
    # Row count and fraction
    param_summary = sample_df[param].value_counts().reset_index()
    param_summary.columns = ['group', 'row_count']
    param_summary['row_fraction'] = param_summary['row_count'] / param_summary['row_count'].sum()

    # Cell count and fraction based on 'count (templates/reads)'
    cell_counts = sample_df.groupby(param)['count (templates/reads)'].sum().reset_index()
    cell_counts.columns = [param, 'cell_count']
    cell_counts['cell_fraction'] = cell_counts['cell_count'] / cell_counts['cell_count'].sum()

    # Merge summaries
    param_summary = param_summary.merge(cell_counts, left_on='group', right_on=param).drop(columns=[param])

    # Add parameter column
    param_summary['param'] = param

    # Append to list
    all_summaries.append(param_summary)

# Concatenate all summaries
final_summary = pd.concat(all_summaries, ignore_index=True)

# Compute summary for grouping by ['vFamilyName', 'jFamilyName', 'cdr3Length']
grouped_summary = sample_df.groupby(['vFamilyName', 'jFamilyName', 'cdr3Length']).agg(
    row_count=('vFamilyName', 'size'),
    cell_count=('count (templates/reads)', 'sum')
).reset_index()

# Add row_fraction and cell_fraction
grouped_summary['row_fraction'] = grouped_summary['row_count'] / grouped_summary['row_count'].sum()
grouped_summary['cell_fraction'] = grouped_summary['cell_count'] / grouped_summary['cell_count'].sum()

# Add parameter column
grouped_summary['param'] = 'vFamilyName, jFamilyName, cdr3Length'

# Append grouped summary to final summary
final_summary = pd.concat([final_summary, grouped_summary], ignore_index=True)

# Display the final summary
return final_summary

        group  row_count  row_fraction  cell_count  cell_fraction  \
0     TCRBV05       2063      0.125502        2326       0.126180   
1     TCRBV06       2031      0.123555        2265       0.122871   
2     TCRBV07       1950      0.118628        2226       0.120755   
3     TCRBV20       1314      0.079937        1447       0.078496   
4     TCRBV28       1019      0.061991        1144       0.062059   
...       ...        ...           ...         ...            ...   
1422      NaN          5      0.000304           6       0.000325   
1423      NaN          6      0.000365           6       0.000325   
1424      NaN          1      0.000061           1       0.000054   
1425      NaN          1      0.000061           1       0.000054   
1426      NaN          1      0.000061           1       0.000054   

                                     param vFamilyName jFamilyName  cdr3Length  
0                              vFamilyName         NaN         NaN         NaN  
1        

sample_df.type 

In [97]:
type(sample_df)

pandas.DataFrame